In [1]:
from noise import decompress_pickle
import pandas as pd
import numpy as np
import pickle
from training_rf_new_dataset import find_max_value, normalizing, transform_tabular_data, apply_notch, notch_filter
import time
INPUT_DATA_PATH = '../input-data/'
MODEL_RF_PATH_NEW = './models_rf_new/'
MODEL_RF_PATH = './models_rf/'
MODEL_ROCKET_PATH_NEW = './models/new_dataset/'
MODEL_ROCKET_PATH = './models/'

In [83]:
def rf_dataset1_time(signal_type, cycle):
    print(f"Tempo operacional tipo do sinal {signal_type}, {cycle} - Random Forest")
    with open(MODEL_RF_PATH + f'{signal_type}_random_forest_{cycle}_max_values.pkl', 'rb') as f:
        max_list = pickle.load(f)

    X_val = decompress_pickle(INPUT_DATA_PATH + f'folds/{signal_type}/{cycle}/X_val')
    X_val = transform_tabular_data(X_val)
    y_val = decompress_pickle(INPUT_DATA_PATH + f'folds/{signal_type}/{cycle}/y_val')
    X_val['target'] = y_val
    sample = X_val.groupby('target').apply(lambda x: x.sample(n=1)).reset_index(drop=True)
    
    letter_dict = {'A': 0, 'B': 0, 'C': 0, 'Z': 0}
    for i in sample.drop(columns='target').columns:
        letter = i.split('_')[0]
        letter_dict[letter] += 1
    time_dict = {}
    for i in range(10):
        s = pd.DataFrame(sample.iloc[i,:-1]).T
        target = sample.loc[i, 'target']
        v = letter_dict['A']
        v2 = 2*v
        v3 = 3*v
        index = int(s.shape[1] / 4)
        # time
        start = time.time()
        a = s.iloc[:,:v] / max_list[0]
        b = s.iloc[:,v:v2] / max_list[1]
        c = s.iloc[:,v2:v3] / max_list[2]
        z = s.iloc[:,v3:] / max_list[3]
        one_fault_sample = pd.concat([a,b,c,z], axis=1)
        one_fault_sample = apply_notch(notch_filter, one_fault_sample, index)
        with open(MODEL_RF_PATH + f'{signal_type}_random_forest_classifier_{cycle}.pkl', 'rb') as f:
            best_model = pickle.load(f)
        y_pred = best_model.predict(one_fault_sample)
        end = time.time()
        time_dict[target] = end - start
        # print(f"Print o tempo operacional para uma amostra da falta {target} foi {end-start}s.")
    max_time = max(time_dict, key=time_dict.get)
    min_time = min(time_dict, key=time_dict.get)
    print(f"O tipo de falta com maior tempo operacional é a falta {max_time}, com {time_dict[max_time]:.3f}s.")
    print(f"O tipo de falta com menor tempo operacional é a falta {min_time}, com {time_dict[min_time]:.3f}s.")

In [84]:
def rf_dataset2_time(signal_type, cycle):
    print(f"Tempo operacional tipo do sinal {signal_type}, {cycle} - Random Forest")
    with open(MODEL_RF_PATH_NEW + f'{signal_type}_random_forest_{cycle}_max_values.pkl', 'rb') as f:
        max_list = pickle.load(f)
    X_val_flavio = decompress_pickle(INPUT_DATA_PATH + f'folds/{signal_type}/{cycle}/X_val')
    y_val_flavio = decompress_pickle(INPUT_DATA_PATH + f'folds/{signal_type}/{cycle}/y_val')
    X_val_robson = decompress_pickle(INPUT_DATA_PATH + f'folds-robson/{signal_type}/{cycle}/X_val')
    y_val_robson = decompress_pickle(INPUT_DATA_PATH + f'folds-robson/{signal_type}/{cycle}/y_val')
    X_val = pd.concat([X_val_flavio, X_val_robson]).reset_index(drop=True)
    X_val = transform_tabular_data(X_val)
    y_val = np.concatenate([y_val_flavio, y_val_robson])

    X_val['target'] = y_val
    sample = X_val.groupby('target').apply(lambda x: x.sample(n=1)).reset_index(drop=True)

    letter_dict = {'A': 0, 'B': 0, 'C': 0, 'Z': 0}
    for i in sample.drop(columns='target').columns:
        letter = i.split('_')[0]
        letter_dict[letter] += 1
        
    time_dict = {}
    for i in range(10):
        s = pd.DataFrame(sample.iloc[i,:-1]).T
        target = sample.loc[i, 'target']
        v = letter_dict['A']
        v2 = 2*v
        v3 = 3*v
        index = int(s.shape[1] / 4)
        # time
        start = time.time()
        a = s.iloc[:,:v] / max_list[0]
        b = s.iloc[:,v:v2] / max_list[1]
        c = s.iloc[:,v2:v3] / max_list[2]
        z = s.iloc[:,v3:] / max_list[3]
        one_fault_sample = pd.concat([a,b,c,z], axis=1)
        one_fault_sample = apply_notch(notch_filter, one_fault_sample, index)
        with open(MODEL_RF_PATH_NEW + f'{signal_type}_random_forest_classifier_{cycle}.pkl', 'rb') as f:
            best_model = pickle.load(f)
        y_pred = best_model.predict(one_fault_sample)
        end = time.time()
        time_dict[target] = end - start
        # print(f"Print o tempo operacional para uma amostra da falta {target} foi {end-start}s.")
    max_time = max(time_dict, key=time_dict.get)
    min_time = min(time_dict, key=time_dict.get)
    print(f"O tipo de falta com maior tempo operacional é a falta {max_time}, com {time_dict[max_time]:.3f}s.")
    print(f"O tipo de falta com menor tempo operacional é a falta {min_time}, com {time_dict[min_time]:.3f}s.")

In [154]:
def rocket_dataset1_time(transformer_name, features, cycle, signal_type):
    print(f"Tempo operacional para {transformer_name} com {features} e {cycle}")
    with open(MODEL_ROCKET_PATH + f'{transformer_name}_{cycle}_max_values_{features}.pkl', 'rb') as f:
        max_list = pickle.load(f)
    with open(MODEL_ROCKET_PATH + f'{transformer_name}_classifier_{cycle}_{features}.pkl', 'rb') as f:
        classifier = pickle.load(f)
    with open(MODEL_ROCKET_PATH + f'{transformer_name}_{cycle}_{features}.pkl', 'rb') as f:
        transformer = pickle.load(f)

    X_val = decompress_pickle(INPUT_DATA_PATH + f'folds/{signal_type}/{cycle}/X_val')
    X_val = transform_tabular_data(X_val)
    y_val = decompress_pickle(INPUT_DATA_PATH + f'folds/{signal_type}/{cycle}/y_val')
    X_val['target'] = y_val
    sample = X_val.groupby('target').apply(lambda x: x.sample(n=1)).reset_index(drop=True)

    letter_dict = {'A': 0, 'B': 0, 'C': 0, 'Z': 0}
    for i in sample.drop(columns='target').columns:
        letter = i.split('_')[0]
        letter_dict[letter] += 1

    time_dict = {}
    for i in range(10):
        s = pd.DataFrame(sample.iloc[i,:-1]).T
        target = sample.loc[i, 'target']
        v = letter_dict['A']
        v2 = 2*v
        v3 = 3*v
        index = int(s.shape[1] / 4)
        # time
        start = time.time()
        a = (s.iloc[:,:v] / max_list[0]).astype(float)
        b = (s.iloc[:,v:v2] / max_list[1]).astype(float)
        c = (s.iloc[:,v2:v3] / max_list[2]).astype(float)
        z = (s.iloc[:,v3:] / max_list[3]).astype(float)

        a = pd.Series(a.values.flatten())
        b = pd.Series(b.values.flatten())
        c = pd.Series(c.values.flatten())
        z = pd.Series(z.values.flatten())

        dicio = {'A': [], 'B': [], 'C': [], 'Z': []}
        dicio['A'].append(a)
        dicio['B'].append(b)
        dicio['C'].append(c)
        dicio['Z'].append(z)
        new_sample = pd.DataFrame(dicio)
        new_sample = transformer.transform(new_sample)
        y_pred = classifier.predict(new_sample)
        end = time.time()
        time_dict[target] = end - start
    max_time = max(time_dict, key=time_dict.get)
    min_time = min(time_dict, key=time_dict.get)
    print(f"O tipo de falta com maior tempo operacional é a falta {max_time}, com {time_dict[max_time]:.3f}s.")
    print(f"O tipo de falta com menor tempo operacional é a falta {min_time}, com {time_dict[min_time]:.3f}s.")

In [156]:
def rocket_dataset2_time(transformer_name, features, cycle, signal_type):
    print(f"Tempo operacional para {transformer_name} com {features} e {cycle}")
    with open(MODEL_ROCKET_PATH + f'{transformer_name}_{cycle}_max_values_{features}.pkl', 'rb') as f:
        max_list = pickle.load(f)
    with open(MODEL_ROCKET_PATH + f'{transformer_name}_classifier_{cycle}_{features}.pkl', 'rb') as f:
        classifier = pickle.load(f)
    with open(MODEL_ROCKET_PATH + f'{transformer_name}_{cycle}_{features}.pkl', 'rb') as f:
        transformer = pickle.load(f)

    X_val_flavio = decompress_pickle(INPUT_DATA_PATH + f'folds/{signal_type}/{cycle}/X_val')
    y_val_flavio = decompress_pickle(INPUT_DATA_PATH + f'folds/{signal_type}/{cycle}/y_val')
    X_val_robson = decompress_pickle(INPUT_DATA_PATH + f'folds-robson/{signal_type}/{cycle}/X_val')
    y_val_robson = decompress_pickle(INPUT_DATA_PATH + f'folds-robson/{signal_type}/{cycle}/y_val')
    X_val = pd.concat([X_val_flavio, X_val_robson]).reset_index(drop=True)
    X_val = transform_tabular_data(X_val)
    y_val = np.concatenate([y_val_flavio, y_val_robson])

    X_val['target'] = y_val
    sample = X_val.groupby('target').apply(lambda x: x.sample(n=1)).reset_index(drop=True)

    letter_dict = {'A': 0, 'B': 0, 'C': 0, 'Z': 0}
    for i in sample.drop(columns='target').columns:
        letter = i.split('_')[0]
        letter_dict[letter] += 1

    time_dict = {}
    for i in range(10):
        s = pd.DataFrame(sample.iloc[i,:-1]).T
        target = sample.loc[i, 'target']
        v = letter_dict['A']
        v2 = 2*v
        v3 = 3*v
        index = int(s.shape[1] / 4)
        # time
        start = time.time()
        a = (s.iloc[:,:v] / max_list[0]).astype(float)
        b = (s.iloc[:,v:v2] / max_list[1]).astype(float)
        c = (s.iloc[:,v2:v3] / max_list[2]).astype(float)
        z = (s.iloc[:,v3:] / max_list[3]).astype(float)

        a = pd.Series(a.values.flatten())
        b = pd.Series(b.values.flatten())
        c = pd.Series(c.values.flatten())
        z = pd.Series(z.values.flatten())

        dicio = {'A': [], 'B': [], 'C': [], 'Z': []}
        dicio['A'].append(a)
        dicio['B'].append(b)
        dicio['C'].append(c)
        dicio['Z'].append(z)
        new_sample = pd.DataFrame(dicio)
        new_sample = transformer.transform(new_sample)
        y_pred = classifier.predict(new_sample)
        end = time.time()
        time_dict[target] = end - start
    max_time = max(time_dict, key=time_dict.get)
    min_time = min(time_dict, key=time_dict.get)
    print(f"O tipo de falta com maior tempo operacional é a falta {max_time}, com {time_dict[max_time]:.3f}s.")
    print(f"O tipo de falta com menor tempo operacional é a falta {min_time}, com {time_dict[min_time]:.3f}s.")

In [158]:
cycles = ['cycle_1', 'cycle_2', 'cycle_4', 'cycle_8', 'cycle_16', 'cycle_32', 'cycle_64', 'cycle_128']
# Método B
rf_dataset1_time('v', 'cycle_1')
# Método C
rf_dataset2_time('v', 'cycle_1')
# Método D
rf_dataset1_time('i', 'cycle_1')
# Método E
rf_dataset2_time('i', 'cycle_1')
# Método F
rocket_dataset1_time('rocket', '100', 'cycle_1', 'i')
rocket_dataset1_time('minirocket', '100', 'cycle_1', 'i')
# Método G
rocket_dataset2_time('rocket', '100', 'cycle_1', 'i')
rocket_dataset2_time('minirocket', '100', 'cycle_1', 'i')

Tempo operacional tipo do sinal v, cycle_1 - Random Forest
O tipo de falta com maior tempo operacional é a falta ABC, com 0.077s.
O tipo de falta com menor tempo operacional é a falta CT, com 0.071s.
Tempo operacional tipo do sinal v, cycle_1 - Random Forest
O tipo de falta com maior tempo operacional é a falta AB, com 0.154s.
O tipo de falta com menor tempo operacional é a falta ABT, com 0.077s.
Tempo operacional tipo do sinal i, cycle_1 - Random Forest
O tipo de falta com maior tempo operacional é a falta CAT, com 0.082s.
O tipo de falta com menor tempo operacional é a falta BC, com 0.076s.
Tempo operacional tipo do sinal i, cycle_1 - Random Forest
O tipo de falta com maior tempo operacional é a falta ABT, com 0.087s.
O tipo de falta com menor tempo operacional é a falta BC, com 0.076s.
Tempo operacional para rocket com 100 e cycle_1
O tipo de falta com maior tempo operacional é a falta AB, com 0.018s.
O tipo de falta com menor tempo operacional é a falta CT, com 0.011s.
Tempo operac

## Testes

In [ ]:
with open(MODEL_RF_PATH_NEW + f'{signal_type}_random_forest_{cycle}_max_values.pkl', 'rb') as f:
    max_list = pickle.load(f)

X_train_norm = normalizing(X_train, max_list)
X_val_norm = normalizing(X_val, max_list)

X_train_norm = transform_tabular_data(X_train_norm)
X_val_norm = transform_tabular_data(X_val_norm)
index = int(X_train_norm.shape[1] / 4)
X_train_norm = apply_notch(notch_filter, X_train_norm, index)
X_val_norm = apply_notch(notch_filter, X_val_norm, index)

In [86]:
with open(MODEL_RF_PATH + f'{signal_type}_random_forest_{cycle}_max_values.pkl', 'rb') as f:
    max_list_rf = pickle.load(f)

X_val = decompress_pickle(INPUT_DATA_PATH + f'folds/{signal_type}/{cycle}/X_val')
X_val_trans = transform_tabular_data(X_val)
y_val = decompress_pickle(INPUT_DATA_PATH + f'folds/{signal_type}/{cycle}/y_val')
X_val_trans['target'] = y_val
sample = X_val_trans.groupby('target').apply(lambda x: x.sample(n=1)).reset_index(drop=True)

In [94]:
sample

,A__0,A__1,A__2,A__3,A__4,A__5,A__6,A__7,A__8,A__9,...,Z__63,Z__64,Z__65,Z__66,Z__67,Z__68,Z__69,Z__70,Z__71,target
0,57813.896852,50020.652105,42057.886046,33315.227750,23601.656985,15003.748787,9500.225018,5593.887255,-6663.726372,-33755.731486,...,-2.349139e+01,-2.498653e+01,-2.561058e+01,-2.596339e+01,-2.620627e+01,-2.616457e+01,-2.607087e+01,-2.642824e+01,-2.849918e+01,AB
1,-292011.671604,-282210.651890,-273510.406582,-265916.033896,-262145.951247,-260201.219743,-259989.648032,-260294.785472,-259239.537791,-256555.747165,...,-5.124077e-07,-5.141967e-07,-4.913957e-07,-4.427695e-07,-3.749861e-07,-3.017487e-07,-2.378019e-07,-1.941328e-07,-1.746400e-07,ABC
2,66111.773068,57850.819960,51859.610123,46069.494599,38920.785080,31481.603139,27264.060262,25333.517002,24997.513551,25341.397862,...,1.161447e+05,1.243046e+05,1.322225e+05,1.403735e+05,1.477567e+05,1.550721e+05,1.623148e+05,1.691064e+05,1.742138e+05,ABT
3,402976.526999,408018.373792,412911.180929,417421.905563,421619.119970,425568.313601,429259.229813,432930.699301,115479.909714,8867.556573,...,-4.683137e+05,-4.522412e+05,-4.459826e+05,-4.511608e+05,-4.503333e+05,-4.168278e+05,-3.518942e+05,-2.850034e+05,-2.358667e+05,AT
4,147651.649340,157694.121051,168261.113315,178400.234954,188671.069779,198535.822123,208564.292795,218659.735253,228174.757423,237814.542443,...,7.209417e+00,4.756777e+00,2.472011e+00,4.487607e-01,-1.252090e+00,-2.549441e+00,-3.624633e+00,-4.488364e+00,-5.320861e+00,BC
5,405145.643023,371321.518246,353972.651058,336326.526500,323046.240626,311973.980594,297822.006611,280070.525563,261589.397244,247496.458391,...,-1.876517e+05,-1.876879e+05,-2.017216e+05,-2.323895e+05,-2.689759e+05,-3.018749e+05,-3.263300e+05,-3.391737e+05,-3.421455e+05,BCT
6,278289.487559,248009.349463,226550.984463,223147.603093,234419.912538,252972.718323,255898.158401,248005.985580,234231.488264,215062.673590,...,-2.720604e+05,-2.808517e+05,-2.776479e+05,-2.551674e+05,-2.354083e+05,-2.482500e+05,-2.865893e+05,-3.144579e+05,-3.187258e+05,BT
7,-132259.788391,-121158.499477,-104858.543140,-86085.604240,-67011.985720,-49009.628359,-34753.551086,-23216.844205,-14420.817241,-8512.140000,...,1.856373e+01,1.863615e+01,1.873045e+01,1.890752e+01,1.926082e+01,1.974245e+01,2.034461e+01,2.091672e+01,2.145501e+01,CA
8,225674.888829,139198.167989,136999.734234,133992.493300,130970.371435,127603.167902,124386.030175,121168.447313,117875.613105,115066.712545,...,3.571931e+05,3.718403e+05,3.644457e+05,3.268912e+05,2.698415e+05,2.112990e+05,1.737427e+05,1.601999e+05,1.621051e+05,CAT
9,-338701.620101,-348469.371571,-358421.289209,-367129.971272,-375618.529437,-384275.328067,-393112.132226,-401366.321372,-408778.161166,-416559.261064,...,-2.932832e+05,-2.903170e+05,-2.885652e+05,-2.893183e+05,-2.910372e+05,-2.924453e+05,-2.943117e+05,-2.945355e+05,-2.951359e+05,CT


In [87]:
letter_dict = {'A': 0, 'B': 0, 'C': 0, 'Z': 0}
for i in sample.drop(columns='target').columns:
    letter = i.split('_')[0]
    letter_dict[letter] += 1

In [93]:
for i in range(10):
    s = pd.DataFrame(sample.iloc[i,:-1]).T
    target = sample.loc[i, 'target']
    v = letter_dict['A']
    v2 = 2*v
    v3 = 3*v
    index = int(s.shape[1] / 4)
    # time
    start = time.time()
    a = s.iloc[:,:v] / max_list[0]
    b = s.iloc[:,v:v2] / max_list[1]
    c = s.iloc[:,v2:v3] / max_list[2]
    z = s.iloc[:,v3:] / max_list[3]
    one_fault_sample = pd.concat([a,b,c,z], axis=1)
    one_fault_sample = apply_notch(notch_filter, one_fault_sample, index)
    with open(MODEL_RF_PATH + f'{signal_type}_random_forest_classifier_{cycle}.pkl', 'rb') as f:
        best_model = pickle.load(f)
    y_pred = best_model.predict(one_fault_sample)
    end = time.time()
    print(f"Print o tempo operacional para uma amostra da falta {target} foi {end-start}s.")

Print o tempo operacional para uma amostra da falta AB foi 0.0654299259185791s.
Print o tempo operacional para uma amostra da falta ABC foi 0.04073643684387207s.
Print o tempo operacional para uma amostra da falta ABT foi 0.039031982421875s.
Print o tempo operacional para uma amostra da falta AT foi 0.02826976776123047s.
Print o tempo operacional para uma amostra da falta BC foi 0.03062295913696289s.
Print o tempo operacional para uma amostra da falta BCT foi 0.030745983123779297s.
Print o tempo operacional para uma amostra da falta BT foi 0.04047346115112305s.
Print o tempo operacional para uma amostra da falta CA foi 0.03576254844665527s.
Print o tempo operacional para uma amostra da falta CAT foi 0.031641244888305664s.
Print o tempo operacional para uma amostra da falta CT foi 0.03338789939880371s.


In [92]:
s

,A__0,A__1,A__2,A__3,A__4,A__5,A__6,A__7,A__8,A__9,...,Z__62,Z__63,Z__64,Z__65,Z__66,Z__67,Z__68,Z__69,Z__70,Z__71
9,-338701.620101,-348469.371571,-358421.289209,-367129.971272,-375618.529437,-384275.328067,-393112.132226,-401366.321372,-408778.161166,-416559.261064,...,-296745.347938,-293283.153506,-290317.008587,-288565.173771,-289318.270744,-291037.227482,-292445.307694,-294311.653042,-294535.456118,-295135.940224


In [91]:
index = int(one_fault_sample.shape[1] / 4)
apply_notch(notch_filter, one_fault_sample, index)

,0,1,2,3,4,5,6,7,8,9,...,278,279,280,281,282,283,284,285,286,287
0,-0.132882,-0.150614,-0.167551,-0.181995,-0.195317,-0.208039,-0.220198,-0.230894,-0.239887,-0.248738,...,-0.105281,-0.100883,-0.097199,-0.094838,-0.094885,-0.095813,-0.096422,-0.097422,-0.096901,-0.096765


In [62]:
X_val.iloc[:,letter_dict['C']:letter_dict['C']+letter_dict['Z']] / max_list[3]

,B__0,B__1,B__2,B__3,B__4,B__5,B__6,B__7,B__8,B__9,...,B__62,B__63,B__64,B__65,B__66,B__67,B__68,B__69,B__70,B__71
535,-0.027222,-0.024164,-0.021614,-0.019286,-0.017466,-0.014585,-0.010707,-0.006450,0.002502,0.013539,...,0.042450,0.075430,0.094111,0.100483,0.101332,0.103016,0.105806,0.107790,0.104827,0.099056
449,0.168732,0.166426,0.164727,0.165608,0.176874,0.199676,0.200415,0.164296,0.136194,0.113864,...,0.161146,0.153101,0.141631,0.128634,0.120239,0.125401,0.139167,0.147007,0.142671,0.131206
921,0.211687,0.210484,0.209460,0.207898,0.206996,0.204948,0.202986,0.201067,0.199203,0.198288,...,-0.020115,-0.022648,-0.025842,-0.030098,-0.033719,-0.036767,-0.040181,-0.043822,-0.047614,-0.051800
537,-0.155157,-0.146054,-0.136290,-0.127087,-0.117425,-0.107762,-0.097773,-0.088076,-0.078448,-0.068585,...,0.369519,0.373808,0.377755,0.381577,0.384965,0.387928,0.391479,0.393748,0.396235,0.398953
155,0.302932,0.309915,0.316552,0.322583,0.329387,0.334876,0.340488,0.346264,0.351267,0.356124,...,0.287147,0.280686,0.273487,0.265800,0.257787,0.250515,0.242322,0.234446,0.225376,0.217053
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
762,0.016421,0.015227,0.015430,0.015609,0.015094,0.014773,0.015268,0.016693,0.016578,0.016033,...,-0.027935,-0.021000,-0.009611,-0.001017,0.007114,0.012505,0.014638,0.015309,0.015841,0.016488
458,0.054950,0.063296,0.070805,0.078165,0.085097,0.092305,0.097734,0.101879,0.106195,0.110095,...,0.277174,0.281132,0.286045,0.287260,0.277470,0.254513,0.231108,0.226078,0.240803,0.263180
50,-0.282294,-0.289940,-0.297113,-0.303136,-0.310172,-0.316871,-0.322270,-0.328788,-0.334532,-0.340365,...,-0.308718,-0.301765,-0.295087,-0.287729,-0.280848,-0.273397,-0.265593,-0.258440,-0.249694,-0.242171
828,0.449191,0.448726,0.447642,0.446015,0.444056,0.442453,0.440753,0.438841,0.437489,0.435865,...,0.102853,0.092650,0.083160,0.072257,0.062343,0.051165,0.040151,0.029170,0.017722,0.006127


In [38]:
288/4

72.0

In [37]:
max_list

[826489.2589167032, 825154.9138621712, 826350.4750070778, 1112191.2669864993]

In [ ]:
y_val = decompress_pickle(INPUT_DATA_PATH + f'folds/{signal_type}/{cycle}/y_val')
X_val_norm['target'] = y_val

In [18]:
with open(MODEL_RF_PATH + f'{signal_type}_random_forest_{cycle}_max_values.pkl', 'rb') as f:
    max_list = pickle.load(f)

X_val = decompress_pickle(INPUT_DATA_PATH + f'folds/{signal_type}/{cycle}/X_val')
X_val_norm = normalizing(X_val, max_list)
X_val_norm = transform_tabular_data(X_val_norm)
index = int(X_val_norm.shape[1] / 4)
X_val_norm = apply_notch(notch_filter, X_val_norm, index)
y_val = decompress_pickle(INPUT_DATA_PATH + f'folds/{signal_type}/{cycle}/y_val')
X_val_norm['target'] = y_val
sample = X_val_norm.groupby('target').apply(lambda x: x.sample(n=1)).reset_index(drop=True)

In [23]:
pd.DataFrame(sample.iloc[0,:-1]).T

,0,1,2,3,4,5,6,7,8,9,...,278,279,280,281,282,283,284,285,286,287
0,0.56482,0.533224,0.506226,0.474744,0.427898,0.379949,0.35636,0.344965,0.337588,0.327482,...,-0.000011,-0.00001,-0.000009,-0.000009,-0.000009,-0.000008,-0.000008,-0.000007,-0.000007,-0.000006


In [30]:
sample.loc[0,'target']

'AB'

In [27]:
pd.DataFrame(sample.iloc[1,:]).T

,0,1,2,3,4,5,6,7,8,9,...,279,280,281,282,283,284,285,286,287,target
1,-0.466291,-0.449693,-0.433694,-0.419223,-0.404888,-0.390422,-0.376202,-0.360664,-0.346415,-0.330745,...,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,ABC


In [26]:
with open(MODEL_RF_PATH + f'{signal_type}_random_forest_classifier_{cycle}.pkl', 'rb') as f:
    best_model = pickle.load(f)

y_pred = best_model.predict(pd.DataFrame(sample.iloc[1,:-1]).T)
print(y_pred)

['ABC']


# Generate Matrix from saved models

In [2]:
def generate_confusion_matrix(y_val, y_pred, image_path='', filename='', title='', colorscale='blues',
                              width=500, height=500):
    data = {'Real':    y_val,
            'Predito': y_pred}
    df = pd.DataFrame(data, columns=['Real','Predito'])
    confusion_matrix = pd.crosstab(df['Real'], df['Predito'], rownames=['Real'],
                                   colnames=['Predito'], margins = True)
    cm = confusion_matrix.drop('All', axis=1).drop('All', axis=0)

    # Inverte rows because create_annotated_heatmap creates matrix in inverted order
    c = cm.values
    x = list(cm.index)
    y = x
    c_text = [[str(y) for y in x] for x in c]

    # fig = ff.create_annotated_heatmap(c, x=x, y=y, annotation_text=c_text, colorscale=colorscale)

    # # add title
    # fig.update_layout(title_text=f'<i><b>Matriz de Confusão {title}</b></i>',
    #                   title_x=0.5, autosize=False, width=width, height=height,)

    # # add custom xaxis title
    # fig.add_annotation(dict(font=dict(color="black",size=14), x=0.5, y=-0.12, showarrow=False,
    #                         text="Valores Preditos", xref="paper", yref="paper"))

    # fig.add_annotation(dict(font=dict(color="black",size=14), x=-0.2, y=0.5, textangle=270,
    #                         showarrow=False, text="Valores Reais", xref="paper", yref="paper"))
    # fig.write_image(image_path + filename + '.svg')
    return c, c_text, x, cm

In [54]:
signal_type, cycle_str, name = 'i', 'cycle_128', 'minirocket'
features, den, method, cycle, nome_cap  = '500', '128', 'G', '1/128', 'MiniRocket'
n1, n2 = 38, 36
# X_train = decompress_pickle(INPUT_DATA_PATH + f'folds/{signal_type}/{cycle_str}/X_train')
# y_train = decompress_pickle(INPUT_DATA_PATH + f'folds/{signal_type}/{cycle_str}/y_train')
# X_val = decompress_pickle(INPUT_DATA_PATH + f'folds/{signal_type}/{cycle_str}/X_val')
# y_val = decompress_pickle(INPUT_DATA_PATH + f'folds/{signal_type}/{cycle_str}/y_val')
X_val_flavio = decompress_pickle(INPUT_DATA_PATH + f'folds/{signal_type}/{cycle_str}/X_val')
y_val_flavio = decompress_pickle(INPUT_DATA_PATH + f'folds/{signal_type}/{cycle_str}/y_val')
X_val_robson = decompress_pickle(INPUT_DATA_PATH + f'folds-robson/{signal_type}/{cycle_str}/X_val')
y_val_robson = decompress_pickle(INPUT_DATA_PATH + f'folds-robson/{signal_type}/{cycle_str}/y_val')
X_val = pd.concat([X_val_flavio, X_val_robson]).reset_index(drop=True)
y_val = np.concatenate([y_val_flavio, y_val_robson])

with open(MODEL_ROCKET_PATH_NEW + f'{name}_{cycle_str}_max_values_{features}.pkl', 'rb') as f:
    max_list = pickle.load(f)
with open(MODEL_ROCKET_PATH_NEW + f'{name}_classifier_{cycle_str}_{features}.pkl', 'rb') as f:
    best_model = pickle.load(f)
with open(MODEL_ROCKET_PATH_NEW + f'{name}_{cycle_str}_{features}.pkl', 'rb') as f:
    transformer = pickle.load(f)
    
# X_train_norm = normalizing(X_train, max_list)
X_val_norm = normalizing(X_val, max_list)
X_val_transform = transformer.transform(X_val_norm)
# X_train_transform = transformer.transform(X_train_norm)

In [55]:
y_pred = best_model.predict(X_val_transform)
_, _, _, cm = generate_confusion_matrix(y_val, y_pred)

In [56]:
ct_ = list(cm.loc['CT',:].values)
ct_ = [str(v) if v != 0 else '' for v in ct_]
ct_ = [v + '/' + str(n1) if v != '' else v for v in ct_]
ct_ = [v + r"\end{tabular} " if v != '' else v for v in ct_]
ct = list(np.round((cm.loc['CT',:] / n1) * 100, 2).values)
ct = [str(v) if v != 0 else '' for v in ct]
ct = [v.replace('.', ',') for v in ct]
ct = [r" \begin{tabular}[c]{@{}c@{}}" + v + r"\%\\ " if v != '' else v for v in ct]

cat_ = list(cm.loc['CAT',:].values)
cat_ = [str(v) if v != 0 else '' for v in cat_]
cat_ = [v + '/' + str(n2) if v != '' else v for v in cat_]
cat_ = [v + r"\end{tabular} " if v != '' else v for v in cat_]
cat = list(np.round((cm.loc['CAT',:] / n2) * 100, 2).values)
cat = [str(v) if v != 0 else '' for v in cat]
cat = [v.replace('.', ',') for v in cat]
cat = [r" \begin{tabular}[c]{@{}c@{}}" + v + r"\%\\ " if v != '' else v for v in cat]

ca_ = list(cm.loc['CA',:].values)
ca_ = [str(v) if v != 0 else '' for v in ca_]
ca_ = [v + '/' + str(n1) if v != '' else v for v in ca_]
ca_ = [v + r"\end{tabular} " if v != '' else v for v in ca_]
ca = list(np.round((cm.loc['CA',:] / n1) * 100, 2).values)
ca = [str(v) if v != 0 else '' for v in ca]
ca = [v.replace('.', ',') for v in ca]
ca = [r" \begin{tabular}[c]{@{}c@{}}" + v + r"\%\\ " if v != '' else v for v in ca]

bt_ = list(cm.loc['BT',:].values)
bt_ = [str(v) if v != 0 else '' for v in bt_]
bt_ = [v + '/' + str(n1) if v != '' else v for v in bt_]
bt_ = [v + r"\end{tabular} " if v != '' else v for v in bt_]
bt = list(np.round((cm.loc['BT',:] / n1) * 100, 2).values)
bt = [str(v) if v != 0 else '' for v in bt]
bt = [v.replace('.', ',') for v in bt]
bt = [r" \begin{tabular}[c]{@{}c@{}}" + v + r"\%\\ " if v != '' else v for v in bt]

bct_ = list(cm.loc['BCT',:].values)
bct_ = [str(v) if v != 0 else '' for v in bct_]
bct_ = [v + '/' + str(n1) if v != '' else v for v in bct_]
bct_ = [v + r"\end{tabular} " if v != '' else v for v in bct_]
bct = list(np.round((cm.loc['BCT',:] / n1) * 100, 2).values)
bct = [str(v) if v != 0 else '' for v in bct]
bct = [v.replace('.', ',') for v in bct]
bct = [r" \begin{tabular}[c]{@{}c@{}}" + v + r"\%\\ " if v != '' else v for v in bct]

bc_ = list(cm.loc['BC',:].values)
bc_ = [str(v) if v != 0 else '' for v in bc_]
bc_ = [v + '/' + str(n1) if v != '' else v for v in bc_]
bc_ = [v + r"\end{tabular} " if v != '' else v for v in bc_]
bc = list(np.round((cm.loc['BC',:] / n1) * 100, 2).values)
bc = [str(v) if v != 0 else '' for v in bc]
bc = [v.replace('.', ',') for v in bc]
bc = [r" \begin{tabular}[c]{@{}c@{}}" + v + r"\%\\ " if v != '' else v for v in bc]

at_ = list(cm.loc['AT',:].values)
at_ = [str(v) if v != 0 else '' for v in at_]
at_ = [v + '/' + str(n1) if v != '' else v for v in at_]
at_ = [v + r"\end{tabular} " if v != '' else v for v in at_]
at = list(np.round((cm.loc['AT',:] / n1) * 100, 2).values)
at = [str(v) if v != 0 else '' for v in at]
at = [v.replace('.', ',') for v in at]
at = [r" \begin{tabular}[c]{@{}c@{}}" + v + r"\%\\ " if v != '' else v for v in at]

abt_ = list(cm.loc['ABT',:].values)
abt_ = [str(v) if v != 0 else '' for v in abt_]
abt_ = [v + '/' + str(n1) if v != '' else v for v in abt_]
abt_ = [v + r"\end{tabular} " if v != '' else v for v in abt_]
abt = list(np.round((cm.loc['ABT',:] / n1) * 100, 2).values)
abt = [str(v) if v != 0 else '' for v in abt]
abt = [v.replace('.', ',') for v in abt]
abt = [r" \begin{tabular}[c]{@{}c@{}}" + v + r"\%\\ " if v != '' else v for v in abt]

abc_ = list(cm.loc['ABC',:].values)
abc_ = [str(v) if v != 0 else '' for v in abc_]
abc_ = [v + '/' + str(n2) if v != '' else v for v in abc_]
abc_ = [v + r"\end{tabular} " if v != '' else v for v in abc_]
abc = list(np.round((cm.loc['ABC',:] / n2) * 100, 2).values)
abc = [str(v) if v != 0 else '' for v in abc]
abc = [v.replace('.', ',') for v in abc]
abc = [r" \begin{tabular}[c]{@{}c@{}}" + v + r"\%\\ " if v != '' else v for v in abc]

ab_ = list(cm.loc['AB',:].values)
ab_ = [str(v) if v != 0 else '' for v in ab_]
ab_ = [v + '/' + str(n1) if v != '' else v for v in ab_]
ab_ = [v + r"\end{tabular} " if v != '' else v for v in ab_]
ab = list(np.round((cm.loc['AB',:] / n1) * 100, 2).values)
ab = [str(v) if v != 0 else '' for v in ab]
ab = [v.replace('.', ',') for v in ab]
ab = [r" \begin{tabular}[c]{@{}c@{}}" + v + r"\%\\ " if v != '' else v for v in ab]

In [57]:
s = rf'''
A Figura~\ref{{cm_{den}_ciclo_{method}_{name}}} apresenta a matriz de confusão gerada pelo método {method} treinado com sinais com {cycle} ciclo pós falta e extração de \textit{{features}} usando \textit{{{nome_cap}}}.

\begin{{table}}[H]
    \centering
    \caption{{Matriz de confusão do método {method} com {cycle} ciclo pós falta e extração de características com \textit{{{nome_cap}}}.}}
    \label{{cm_{den}_ciclo_{method}_{name}}}
    \begin{{adjustbox}}{{width=0.75\columnwidth,center}}
    \renewcommand{{\arraystretch}}{{1.5}}
    \begin{{tabular}}{{cccccccccccc}}
    \hline
    \multicolumn{{1}}{{l}}{{}} & \multicolumn{{11}}{{c}}{{\textbf{{Valores preditos}}}} \\ \hline
    \multirow{{11}}{{*}}{{\rotatebox[origin=c]{{90}}{{\textbf{{Valores reais}}}}}} &  & AB & ABC & ABT & AT & BC & BCT & BT & CA & CAT & CT \\ \cline{{2-12}} 
     & AB &{ab[0]}{ab_[0]}&{ab[1]}{ab_[1]}&{ab[2]}{ab_[2]}&{ab[3]}{ab_[3]}&{ab[4]}{ab_[4]}&{ab[5]}{ab_[5]}&{ab[6]}{ab_[6]}&{ab[7]}{ab_[7]}&{ab[8]}{ab_[8]}&{ab[9]}{ab_[9]}\\ \cline{{2-12}} 
     & ABC &{abc[0]}{abc_[0]}&{abc[1]}{abc_[1]}&{abc[2]}{abc_[2]}&{abc[3]}{abc_[3]}&{abc[4]}{abc_[4]}&{abc[5]}{abc_[5]}&{abc[6]}{abc_[6]}&{abc[7]}{abc_[7]}&{abc[8]}{abc_[8]}&{abc[9]}{abc_[9]}\\ \cline{{2-12}} 
     & ABT &{abt[0]}{abt_[0]}&{abt[1]}{abt_[1]}&{abt[2]}{abt_[2]}&{abt[3]}{abt_[3]}&{abt[4]}{abt_[4]}&{abt[5]}{abt_[5]}&{abt[6]}{abt_[6]}&{abt[7]}{abt_[7]}&{abt[8]}{abt_[8]}&{abt[9]}{abt_[9]}\\ \cline{{2-12}} 
     & AT &{at[0]}{at_[0]}&{at[1]}{at_[1]}&{at[2]}{at_[2]}&{at[3]}{at_[3]}&{at[4]}{at_[4]}&{at[5]}{at_[5]}&{at[6]}{at_[6]}&{at[7]}{at_[7]}&{at[8]}{at_[8]}&{at[9]}{at_[9]}\\ \cline{{2-12}} 
     & BC &{bc[0]}{bc_[0]}&{bc[1]}{bc_[1]}&{bc[2]}{bc_[2]}&{bc[3]}{bc_[3]}&{bc[4]}{bc_[4]}&{bc[5]}{bc_[5]}&{bc[6]}{bc_[6]}&{bc[7]}{bc_[7]}&{bc[8]}{bc_[8]}&{bc[9]}{bc_[9]}\\ \cline{{2-12}} 
     & BCT &{bct[0]}{bct_[0]}&{bct[1]}{bct_[1]}&{bct[2]}{bct_[2]}&{bct[3]}{bct_[3]}&{bct[4]}{bct_[4]}&{bct[5]}{bct_[5]}&{bct[6]}{bct_[6]}&{bct[7]}{bct_[7]}&{bct[8]}{bct_[8]}&{bct[9]}{bct_[9]}\\ \cline{{2-12}} 
     & BT &{bt[0]}{bt_[0]}&{bt[1]}{bt_[1]}&{bt[2]}{bt_[2]}&{bt[3]}{bt_[3]}&{bt[4]}{bt_[4]}&{bt[5]}{bt_[5]}&{bt[6]}{bt_[6]}&{bt[7]}{bt_[7]}&{bt[8]}{bt_[8]}&{bt[9]}{bt_[9]}\\ \cline{{2-12}} 
     & CA &{ca[0]}{ca_[0]}&{ca[1]}{ca_[1]}&{ca[2]}{ca_[2]}&{ca[3]}{ca_[3]}&{ca[4]}{ca_[4]}&{ca[5]}{ca_[5]}&{ca[6]}{ca_[6]}&{ca[7]}{ca_[7]}&{ca[8]}{ca_[8]}&{ca[9]}{ca_[9]}\\ \cline{{2-12}} 
     & CAT &{cat[0]}{cat_[0]}&{cat[1]}{cat_[1]}&{cat[2]}{cat_[2]}&{cat[3]}{cat_[3]}&{cat[4]}{cat_[4]}&{cat[5]}{cat_[5]}&{cat[6]}{cat_[6]}&{cat[7]}{cat_[7]}&{cat[8]}{cat_[8]}&{cat[9]}{cat_[9]}\\ \cline{{2-12}} 
     & CT &{ct[0]}{ct_[0]}&{ct[1]}{ct_[1]}&{ct[2]}{ct_[2]}&{ct[3]}{ct_[3]}&{ct[4]}{ct_[4]}&{ct[5]}{ct_[5]}&{ct[6]}{ct_[6]}&{ct[7]}{ct_[7]}&{ct[8]}{ct_[8]}&{ct[9]}{ct_[9]}\\ \hline
    \end{{tabular}}
    \end{{adjustbox}}
    {{\newline \\ \small Do autor (2021).}}
    \end{{table}}
'''

In [58]:
print(s)


A Figura~\ref{cm_128_ciclo_G_minirocket} apresenta a matriz de confusão gerada pelo método G treinado com sinais com 1/128 ciclo pós falta e extração de \textit{features} usando \textit{MiniRocket}.

\begin{table}[H]
    \centering
    \caption{Matriz de confusão do método G com 1/128 ciclo pós falta e extração de características com \textit{MiniRocket}.}
    \label{cm_128_ciclo_G_minirocket}
    \begin{adjustbox}{width=0.75\columnwidth,center}
    \renewcommand{\arraystretch}{1.5}
    \begin{tabular}{cccccccccccc}
    \hline
    \multicolumn{1}{l}{} & \multicolumn{11}{c}{\textbf{Valores preditos}} \\ \hline
    \multirow{11}{*}{\rotatebox[origin=c]{90}{\textbf{Valores reais}}} &  & AB & ABC & ABT & AT & BC & BCT & BT & CA & CAT & CT \\ \cline{2-12} 
     & AB & \begin{tabular}[c]{@{}c@{}}63,16\%\\ 24/38\end{tabular} & \begin{tabular}[c]{@{}c@{}}10,53\%\\ 4/38\end{tabular} & \begin{tabular}[c]{@{}c@{}}2,63\%\\ 1/38\end{tabular} && \begin{tabular}[c]{@{}c@{}}2,63\%\\ 1/38\end{tabular} 

# Genearate confusion matrix from pictures

In [2]:
from bs4 import BeautifulSoup

In [267]:
model, den, method, cycle, signal, name, nome_cap = 'cm', '128', 'G', '1/128', 'i', 'minirocket', 'MiniRocket'
n1, n2 = 38, 36

In [268]:
with open(f'./figs_cm/new_dataset/cycle_{den}_{name}.svg', 'rb') as f:
    svg = f.read()

In [269]:
soup = BeautifulSoup(svg, 'html.parser')
values = soup.find_all(class_="annotation")[:-2]
values = [int(v.text) for v in values]
ct_ = values[0:10]
ct_ = [str(v) if v != 0 else '' for v in ct_]
ct_ = [v + '/' + str(n1) if v != '' else v for v in ct_]
ct_ = [v + r"\end{tabular} " if v != '' else v for v in ct_]
ct = list(np.round((np.array(values)[0:10] / n1) * 100, 2))
ct = [str(v) if v != 0 else '' for v in ct]
ct = [v.replace('.', ',') for v in ct]
ct = [r" \begin{tabular}[c]{@{}c@{}}" + v + r"\%\\ " if v != '' else v for v in ct]

cat_ = values[10:20]
cat_ = [str(v) if v != 0 else '' for v in cat_]
cat_ = [v + '/' + str(n2) if v != '' else v for v in cat_]
cat_ = [v + r"\end{tabular} " if v != '' else v for v in cat_]
cat = list(np.round((np.array(values)[10:20] / n2) * 100, 2))
cat = [str(v) if v != 0 else '' for v in cat]
cat = [v.replace('.', ',') for v in cat]
cat = [r" \begin{tabular}[c]{@{}c@{}}" + v + r"\%\\ " if v != '' else v for v in cat]

ca_ = values[20:30]
ca_ = [str(v) if v != 0 else '' for v in ca_]
ca_ = [v + '/' + str(n1) if v != '' else v for v in ca_]
ca_ = [v + r"\end{tabular} " if v != '' else v for v in ca_]
ca = list(np.round((np.array(values)[20:30] / n1) * 100, 2))
ca = [str(v) if v != 0 else '' for v in ca]
ca = [v.replace('.', ',') for v in ca]
ca = [r" \begin{tabular}[c]{@{}c@{}}" + v + r"\%\\ " if v != '' else v for v in ca]

bt_ = values[30:40]
bt_ = [str(v) if v != 0 else '' for v in bt_]
bt_ = [v + '/' + str(n1) if v != '' else v for v in bt_]
bt_ = [v + r"\end{tabular} " if v != '' else v for v in bt_]
bt = list(np.round((np.array(values)[30:40] / n1) * 100, 2))
bt = [str(v) if v != 0 else '' for v in bt]
bt = [v.replace('.', ',') for v in bt]
bt = [r" \begin{tabular}[c]{@{}c@{}}" + v + r"\%\\ " if v != '' else v for v in bt]

bct_ = values[40:50]
bct_ = [str(v) if v != 0 else '' for v in bct_]
bct_ = [v + '/' + str(n1) if v != '' else v for v in bct_]
bct_ = [v + r"\end{tabular} " if v != '' else v for v in bct_]
bct = list(np.round((np.array(values)[40:50] / n1) * 100, 2))
bct = [str(v) if v != 0 else '' for v in bct]
bct = [v.replace('.', ',') for v in bct]
bct = [r" \begin{tabular}[c]{@{}c@{}}" + v + r"\%\\ " if v != '' else v for v in bct]

bc_ = values[50:60]
bc_ = [str(v) if v != 0 else '' for v in bc_]
bc_ = [v + '/' + str(n1) if v != '' else v for v in bc_]
bc_ = [v + r"\end{tabular} " if v != '' else v for v in bc_]
bc = list(np.round((np.array(values)[50:60] / n1) * 100, 2))
bc = [str(v) if v != 0 else '' for v in bc]
bc = [v.replace('.', ',') for v in bc]
bc = [r" \begin{tabular}[c]{@{}c@{}}" + v + r"\%\\ " if v != '' else v for v in bc]

at_ = values[60:70]
at_ = [str(v) if v != 0 else '' for v in at_]
at_ = [v + '/' + str(n1) if v != '' else v for v in at_]
at_ = [v + r"\end{tabular} " if v != '' else v for v in at_]
at = list(np.round((np.array(values)[60:70] / n1) * 100, 2))
at = [str(v) if v != 0 else '' for v in at]
at = [v.replace('.', ',') for v in at]
at = [r" \begin{tabular}[c]{@{}c@{}}" + v + r"\%\\ " if v != '' else v for v in at]

abt_ = values[70:80]
abt_ = [str(v) if v != 0 else '' for v in abt_]
abt_ = [v + '/' + str(n1) if v != '' else v for v in abt_]
abt_ = [v + r"\end{tabular} " if v != '' else v for v in abt_]
abt = list(np.round((np.array(values)[70:80] / n1) * 100, 2))
abt = [str(v) if v != 0 else '' for v in abt]
abt = [v.replace('.', ',') for v in abt]
abt = [r" \begin{tabular}[c]{@{}c@{}}" + v + r"\%\\ " if v != '' else v for v in abt]

abc_ = values[80:90]
abc_ = [str(v) if v != 0 else '' for v in abc_]
abc_ = [v + '/' + str(n2) if v != '' else v for v in abc_]
abc_ = [v + r"\end{tabular} " if v != '' else v for v in abc_]
abc = list(np.round((np.array(values)[80:90] / n2) * 100, 2))
abc = [str(v) if v != 0 else '' for v in abc]
abc = [v.replace('.', ',') for v in abc]
abc = [r" \begin{tabular}[c]{@{}c@{}}" + v + r"\%\\ " if v != '' else v for v in abc]

ab_ = values[90:100]
ab_ = [str(v) if v != 0 else '' for v in ab_]
ab_ = [v + '/' + str(n1) if v != '' else v for v in ab_]
ab_ = [v + r"\end{tabular} " if v != '' else v for v in ab_]
ab = list(np.round((np.array(values)[90:100] / n1) * 100, 2))
ab = [str(v) if v != 0 else '' for v in ab]
ab = [v.replace('.', ',') for v in ab]
ab = [r" \begin{tabular}[c]{@{}c@{}}" + v + r"\%\\ " if v != '' else v for v in ab]

In [270]:
s = rf'''
A Figura~\ref{{{model}_{den}_ciclo_{method}_{name}}} apresenta a matriz de confusão gerada pelo método {method} treinado com sinais com {cycle} ciclo pós falta e extração de \textit{{features}} usando \textit{{{nome_cap}}}.

\begin{{table}}[H]
    \centering
    \caption{{Matriz de confusão do método {method} com {cycle} ciclo pós falta e extração de características com \textit{{{nome_cap}}}.}}
    \label{{{model}_{den}_ciclo_{method}_{name}}}
    \begin{{adjustbox}}{{width=0.8\columnwidth,center}}
    \renewcommand{{\arraystretch}}{{1.5}}
    \begin{{tabular}}{{cccccccccccc}}
    \hline
    \multicolumn{{1}}{{l}}{{}} & \multicolumn{{11}}{{c}}{{\textbf{{Valores preditos}}}} \\ \hline
    \multirow{{11}}{{*}}{{\rotatebox[origin=c]{{90}}{{\textbf{{Valores reais}}}}}} &  & AB & ABC & ABT & AT & BC & BCT & BT & CA & CAT & CT \\ \cline{{2-12}} 
     & AB &{ab[0]}{ab_[0]}&{ab[1]}{ab_[1]}&{ab[2]}{ab_[2]}&{ab[3]}{ab_[3]}&{ab[4]}{ab_[4]}&{ab[5]}{ab_[5]}&{ab[6]}{ab_[6]}&{ab[7]}{ab_[7]}&{ab[8]}{ab_[8]}&{ab[9]}{ab_[9]}\\ \cline{{2-12}} 
     & ABC &{abc[0]}{abc_[0]}&{abc[1]}{abc_[1]}&{abc[2]}{abc_[2]}&{abc[3]}{abc_[3]}&{abc[4]}{abc_[4]}&{abc[5]}{abc_[5]}&{abc[6]}{abc_[6]}&{abc[7]}{abc_[7]}&{abc[8]}{abc_[8]}&{abc[9]}{abc_[9]}\\ \cline{{2-12}} 
     & ABT &{abt[0]}{abt_[0]}&{abt[1]}{abt_[1]}&{abt[2]}{abt_[2]}&{abt[3]}{abt_[3]}&{abt[4]}{abt_[4]}&{abt[5]}{abt_[5]}&{abt[6]}{abt_[6]}&{abt[7]}{abt_[7]}&{abt[8]}{abt_[8]}&{abt[9]}{abt_[9]}\\ \cline{{2-12}} 
     & AT &{at[0]}{at_[0]}&{at[1]}{at_[1]}&{at[2]}{at_[2]}&{at[3]}{at_[3]}&{at[4]}{at_[4]}&{at[5]}{at_[5]}&{at[6]}{at_[6]}&{at[7]}{at_[7]}&{at[8]}{at_[8]}&{at[9]}{at_[9]}\\ \cline{{2-12}} 
     & BC &{bc[0]}{bc_[0]}&{bc[1]}{bc_[1]}&{bc[2]}{bc_[2]}&{bc[3]}{bc_[3]}&{bc[4]}{bc_[4]}&{bc[5]}{bc_[5]}&{bc[6]}{bc_[6]}&{bc[7]}{bc_[7]}&{bc[8]}{bc_[8]}&{bc[9]}{bc_[9]}\\ \cline{{2-12}} 
     & BCT &{bct[0]}{bct_[0]}&{bct[1]}{bct_[1]}&{bct[2]}{bct_[2]}&{bct[3]}{bct_[3]}&{bct[4]}{bct_[4]}&{bct[5]}{bct_[5]}&{bct[6]}{bct_[6]}&{bct[7]}{bct_[7]}&{bct[8]}{bct_[8]}&{bct[9]}{bct_[9]}\\ \cline{{2-12}} 
     & BT &{bt[0]}{bt_[0]}&{bt[1]}{bt_[1]}&{bt[2]}{bt_[2]}&{bt[3]}{bt_[3]}&{bt[4]}{bt_[4]}&{bt[5]}{bt_[5]}&{bt[6]}{bt_[6]}&{bt[7]}{bt_[7]}&{bt[8]}{bt_[8]}&{bt[9]}{bt_[9]}\\ \cline{{2-12}} 
     & CA &{ca[0]}{ca_[0]}&{ca[1]}{ca_[1]}&{ca[2]}{ca_[2]}&{ca[3]}{ca_[3]}&{ca[4]}{ca_[4]}&{ca[5]}{ca_[5]}&{ca[6]}{ca_[6]}&{ca[7]}{ca_[7]}&{ca[8]}{ca_[8]}&{ca[9]}{ca_[9]}\\ \cline{{2-12}} 
     & CAT &{cat[0]}{cat_[0]}&{cat[1]}{cat_[1]}&{cat[2]}{cat_[2]}&{cat[3]}{cat_[3]}&{cat[4]}{cat_[4]}&{cat[5]}{cat_[5]}&{cat[6]}{cat_[6]}&{cat[7]}{cat_[7]}&{cat[8]}{cat_[8]}&{cat[9]}{cat_[9]}\\ \cline{{2-12}} 
     & CT &{ct[0]}{ct_[0]}&{ct[1]}{ct_[1]}&{ct[2]}{ct_[2]}&{ct[3]}{ct_[3]}&{ct[4]}{ct_[4]}&{ct[5]}{ct_[5]}&{ct[6]}{ct_[6]}&{ct[7]}{ct_[7]}&{ct[8]}{ct_[8]}&{ct[9]}{ct_[9]}\\ \hline
    \end{{tabular}}
    \end{{adjustbox}}
    {{\newline \\ \small Do autor (2021).}}
    \end{{table}}
'''

In [271]:
print(s)


A Figura~\ref{cm_128_ciclo_G_minirocket} apresenta a matriz de confusão gerada pelo método G treinado com sinais com 1/128 ciclo pós falta e extração de \textit{features} usando \textit{MiniRocket}.

\begin{table}[H]
    \centering
    \caption{Matriz de confusão do método G com 1/128 ciclo pós falta e extração de características com \textit{MiniRocket}.}
    \label{cm_128_ciclo_G_minirocket}
    \begin{adjustbox}{width=0.8\columnwidth,center}
    \renewcommand{\arraystretch}{1.5}
    \begin{tabular}{cccccccccccc}
    \hline
    \multicolumn{1}{l}{} & \multicolumn{11}{c}{\textbf{Valores preditos}} \\ \hline
    \multirow{11}{*}{\rotatebox[origin=c]{90}{\textbf{Valores reais}}} &  & AB & ABC & ABT & AT & BC & BCT & BT & CA & CAT & CT \\ \cline{2-12} 
     & AB & \begin{tabular}[c]{@{}c@{}}55,26\%\\ 21/38\end{tabular} & \begin{tabular}[c]{@{}c@{}}5,26\%\\ 2/38\end{tabular} & \begin{tabular}[c]{@{}c@{}}5,26\%\\ 2/38\end{tabular} & \begin{tabular}[c]{@{}c@{}}7,89\%\\ 3/38\end{tabular} & \